In [1]:
import pandas as pd
pd.set_option("display.max_rows", None)
pd.set_option('display.max_colwidth', None)
#import os
#import re
import matplotlib.pyplot as plt
#import pickle
from IPython.display import display, Markdown
import seaborn as sns
from math import floor
import ast
import json

#import dbmsbenchmarker
#import bexhoma
from bexhoma import evaluators

%matplotlib inline

In [2]:
path = r"C:\data\benchmarks"

In [3]:
def get_performance(evaluation):
    df = evaluation.get_df_benchmarking()
    df = df.sort_values(['experiment_run', 'client'])
    result = df.groupby('client').agg({
        'Goodput (requests/second)': 'sum',
        'Latency Distribution.Average Latency (microseconds)': 'mean'
    }).reset_index()
    return result

def get_loading_time_max(evaluation):
    with open(path+"/"+code+"/connections.config",'r') as inf:
        connections = ast.literal_eval(inf.read())
        pretty_connections = json.dumps(connections, indent=2)
        #print(pretty_connections)
        connections_sorted = sorted(connections, key=lambda c: c['name'])
        result = dict()
        for c in connections_sorted:
            """
            print(c['name'], 
                  c['timeLoad'], 
                  '[s] for', 
                  c['parameter']['connection_parameter']['loading_parameters']['BENCHBASE_TERMINALS'], 
                  'threads on',
                  c['hostsystem']['node'])
            """
            result[c['name']] = {
                'time_load': c['timeIngesting'],
                'terminals': c['parameter']['connection_parameter']['loading_parameters']['BENCHBASE_TERMINALS'],
                #'target': c['parameter']['connection_parameter']['loading_parameters']['BENCHBASE_TARGET'],
                'pods': c['parameter']['parallelism'],
            }
            #result[c['parameter']['connection_parameter']['loading_parameters']['BENCHBASE_TERMINALS']] = c['timeIngesting']
        df = pd.DataFrame(result).T
        return df['time_load'].max()

def show_summary_monitoring_table(evaluate, component):
    df_monitoring = list()
    ##########
    df = evaluate.get_monitoring_metric(metric='total_cpu_util_s', component=component)
    df = df.max().sort_index() - df.min().sort_index() # compute difference of counter
    #df = df.T.max().sort_index() - df.T.min().sort_index() # compute difference of counter
    df_cleaned = pd.DataFrame(df)
    df_cleaned.columns = ["CPU [CPUs]"]
    if not df_cleaned.empty:
        df_monitoring.append(df_cleaned.copy())
    df = evaluate.get_monitoring_metric(metric='total_cpu_memory', component=component)/1024
    #df = evaluate.get_loading_metrics('total_cpu_memory')/1024
    df = df.max().sort_index()
    df_cleaned = pd.DataFrame(df).round(2)
    df_cleaned.columns = ["Max RAM [Gb]"]
    if not df_cleaned.empty:
        df_monitoring.append(df_cleaned.copy())
    ##########
    df = evaluate.get_monitoring_metric(metric='total_cpu_memory_cached', component=component)/1024
    #df = evaluate.get_loading_metrics('total_cpu_memory_cached')/1024
    df = df.max().sort_index()
    df_cleaned = pd.DataFrame(df)
    df_cleaned.columns = ["Max RAM Cached [Gb]"]
    if not df_cleaned.empty:
        df_monitoring.append(df_cleaned.copy())
    return df_monitoring

def get_monitoring_stream(evaluation):
    df_monitoring = show_summary_monitoring_table(evaluation, "stream")
    if len(df_monitoring) > 0:
        print("\n### Stream SUT")
        df = pd.concat(df_monitoring, axis=1).round(2)
        df = df.reindex(index=evaluators.natural_sort(df.index))
        # If your DataFrame is named df and the index is a string like 'PostgreSQL-1-1-1024-0-1'
        df = df.copy()  # avoid modifying original
        df['client'] = df.index.str.rsplit('-', n=1).str[-1]
        #print(df)
        result = df.groupby('client').agg({
            'CPU [CPUs]': 'sum',
            'Max RAM [Gb]': 'sum',
            'Max RAM Cached [Gb]': 'sum',
        }).reset_index()
        return result


# Container

In [4]:
code = "1750591366"
code="1750618040"
evaluation = evaluators.benchbase(code=code, path=path)

In [5]:
get_performance(evaluation)

,client,Goodput (requests/second),Latency Distribution.Average Latency (microseconds)
0,1,1567.823224,12549.5
1,2,1570.913187,12510.5


In [6]:
get_loading_time_max(evaluation)

66.0

In [7]:
get_monitoring_stream(evaluation)


### Stream SUT


,client,CPU [CPUs],Max RAM [Gb],Max RAM Cached [Gb]
0,1,1750.23,5.37,5.97
1,2,1688.45,5.79,6.78


# Database

In [8]:
code = '1750590405'
code = '1750616828'
evaluation = evaluators.benchbase(code=code, path=path)

In [9]:
get_performance(evaluation)

,client,Goodput (requests/second),Latency Distribution.Average Latency (microseconds)
0,1,1465.633219,13426.5
1,2,1587.769744,12483.5


In [10]:
get_loading_time_max(evaluation)

67.0

In [11]:
get_monitoring_stream(evaluation)


### Stream SUT


,client,CPU [CPUs],Max RAM [Gb],Max RAM Cached [Gb]
0,1,1720.69,3.09,3.66
1,2,1770.50,3.46,4.40


# Schema

In [12]:
code = '1750589482'
code = '1750615583'
evaluation = evaluators.benchbase(code=code, path=path)

In [13]:
get_performance(evaluation)

,client,Goodput (requests/second),Latency Distribution.Average Latency (microseconds)
0,1,1469.156441,13408.0
1,2,1338.416519,14712.0


In [14]:
get_loading_time_max(evaluation)

70.0

In [15]:
get_monitoring_stream(evaluation)


### Stream SUT


,client,CPU [CPUs],Max RAM [Gb],Max RAM Cached [Gb]
0,1,1742.85,3.07,3.65
1,2,1661.71,3.41,4.32
